In [1]:
import sys
import time
import torch
import torch.nn.functional as F
from utils.utils import load_data, mat2tensor, regression_loss
from model.gcn import GCN
from model.gat import GAT
import numpy as np
import dgl

In [2]:
features_list, adjM, labels, train_val_test_idx, dl = load_data('iYO844')
features_list = [mat2tensor(features)
                    for features in features_list]
m_dim = features_list[1].shape[1]

# Set train, val, test index
labels = torch.FloatTensor(labels)
train_idx = train_val_test_idx['train_idx']
train_idx = np.sort(train_idx)
val_idx = train_val_test_idx['val_idx']
val_idx = np.sort(val_idx)
test_idx = train_val_test_idx['test_idx']
test_idx = np.sort(test_idx)

# Build graph
g = dgl.from_scipy(adjM+(adjM.T))
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

# Set model
num_labels = dl.labels_train['num_labels']

In [3]:
net = GCN(g, m_dim, 64, num_labels,
                  2, F.elu, 0.5)

In [5]:
# Test model
net.load_state_dict(torch.load(
    'checkpoint/checkpoint_{}_{}.pth'.format('iYO844', 'gcn')))
net.eval()
test_logits = []
with torch.no_grad():
    logits = net(features_list)
    test_logits = logits[test_idx]
    # dl.gen_file_for_evaluate(test_idx=test_idx,label=pred)
    pred = test_logits.cpu().numpy()
    print(dl.evaluate(pred))

{'km_mse': -8.6028399904935, 'kcat_mse': 0.0}
